In [1]:

# Imports


import os
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from datetime import datetime

from config import *
from data_preprocessing import load_and_prepare_data, split_by_timestamp
from utils import (
    set_seeds,
    scale_numeric_features,
    create_sequences,
    to_python_types
)
from models import NinjaOptimizationAlgorithm, create_lstm_model
from train import objective_function_lstm
from evaluate import evaluate_model


# GPU Safety

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

set_seeds(RANDOM_SEED)
sns.set(style="whitegrid")

print("TensorFlow version:", tf.__version__)
print("GPU available:", len(gpus) > 0)


TensorFlow version: 2.10.1
GPU available: True


In [2]:

# Horizon to k = 60


FORECAST_HORIZON = 60
print(f"\n=== Running k={FORECAST_HORIZON} with SEQUENCE_LENGTH={SEQUENCE_LENGTH} ===")

df = load_and_prepare_data(DATA_PATH, k=FORECAST_HORIZON)

# Fixed timestamp split
t_train_end = df["server_timestamp"].quantile(TRAIN_RATIO)
t_val_end = df["server_timestamp"].quantile(TRAIN_RATIO + VAL_RATIO)

train_df, val_df, test_df = split_by_timestamp(df, t_train_end, t_val_end)

TARGET = "energy_delta_k"

feature_cols = [
    c for c in train_df.columns
    if c not in ["server_timestamp", "energy", TARGET]
]

X_train, y_train = train_df[feature_cols], train_df[TARGET]
X_val, y_val = val_df[feature_cols], val_df[TARGET]
X_test, y_test = test_df[feature_cols], test_df[TARGET]


# Scale X (TRAIN ONLY)

X_train, X_val, X_test, scaler = scale_numeric_features(
    X_train.copy(), X_val.copy(), X_test.copy(), feature_cols
)


# Create sequences


X_train_seq, y_train_seq = create_sequences(
    X_train.values, y_train.values, SEQUENCE_LENGTH
)
X_val_seq, y_val_seq = create_sequences(
    X_val.values, y_val.values, SEQUENCE_LENGTH
)
X_test_seq, y_test_seq = create_sequences(
    X_test.values, y_test.values, SEQUENCE_LENGTH
)

seq_len = X_train_seq.shape[1]
num_feats = X_train_seq.shape[2]

print("Train shape:", X_train_seq.shape)
print("Validation shape:", X_val_seq.shape)
print("Test shape:", X_test_seq.shape)



=== Running k=60 with SEQUENCE_LENGTH=120 ===
Train shape: (2016429, 120, 17)
Validation shape: (431997, 120, 17)
Test shape: (431998, 120, 17)


In [3]:
print("Train range:",
      train_df["server_timestamp"].min(),
      "→",
      train_df["server_timestamp"].max())

print("Validation range:",
      val_df["server_timestamp"].min(),
      "→",
      val_df["server_timestamp"].max())

print("Test range:",
      test_df["server_timestamp"].min(),
      "→",
      test_df["server_timestamp"].max())


Train range: 2021-08-05 12:44:50 → 2021-09-12 04:11:57
Validation range: 2021-09-12 04:11:58 → 2021-11-26 02:11:39
Test range: 2021-11-26 02:11:40 → 2021-12-04 08:17:32


In [4]:
assert train_df["server_timestamp"].max() < val_df["server_timestamp"].min()
assert val_df["server_timestamp"].max() < test_df["server_timestamp"].min()
print("Timestamp split verified: strictly chronological.")


Timestamp split verified: strictly chronological.


In [5]:

# Results folder


timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

experiment_dir = os.path.join(
    RESULTS_DIR,
    f"k60_seq{SEQUENCE_LENGTH}_{timestamp}"
)

os.makedirs(experiment_dir, exist_ok=True)
os.makedirs(os.path.join(experiment_dir, "model"), exist_ok=True)
os.makedirs(os.path.join(experiment_dir, "predictions"), exist_ok=True)
os.makedirs(os.path.join(experiment_dir, "plots"), exist_ok=True)

print("Experiment directory:", experiment_dir)


Experiment directory: C:\Users\AnweshaSingh\anaconda_projects\IEEE_CEP_V6_1min\results\k60_seq120_20260212_152425


In [6]:
# Save sequence tensors
np.save(os.path.join(experiment_dir, "X_train.npy"), X_train_seq)
np.save(os.path.join(experiment_dir, "X_val.npy"), X_val_seq)
np.save(os.path.join(experiment_dir, "X_test.npy"), X_test_seq)

np.save(os.path.join(experiment_dir, "y_train.npy"), y_train_seq)
np.save(os.path.join(experiment_dir, "y_val.npy"), y_val_seq)
np.save(os.path.join(experiment_dir, "y_test.npy"), y_test_seq)

# Save scaler
joblib.dump(scaler, os.path.join(experiment_dir, "scaler.pkl"))

print("Data splits and scaler saved.")


Data splits and scaler saved.


In [7]:
print("Search space:", HYPERPARAMETER_BOUNDS)
print("Train seq shape:", X_train_seq.shape)
print("Validation seq shape:", X_val_seq.shape)


Search space: {'lstm_layers': ([2, 3], 'int'), 'units': ([64, 128], 'categorical'), 'dropout': ([0.3, 0.6], 'float'), 'optimizer': (['adamw'], 'categorical'), 'learning_rate': ([5e-05, 0.0005], 'float_log'), 'batch_size': ([32], 'categorical')}
Train seq shape: (2016429, 120, 17)
Validation seq shape: (431997, 120, 17)


In [8]:
assert len(X_train_seq) > 0, "Empty training sequences!"
assert len(X_val_seq) > 0, "Empty validation sequences!"


In [9]:

# Subset for optimisation


n_train_opt = int(len(X_train_seq) * OPT_SUBSET_RATIO)
n_val_opt = int(len(X_val_seq) * OPT_SUBSET_RATIO)

print("Optimisation subset sizes:", n_train_opt, n_val_opt)

ninja = NinjaOptimizationAlgorithm(
    objective_function=lambda p: objective_function_lstm(
        p,
        X_train_seq[:n_train_opt],
        y_train_seq[:n_train_opt],
        X_val_seq[:n_val_opt],
        y_val_seq[:n_val_opt]
    ),
    bounds=HYPERPARAMETER_BOUNDS,
    n_agents=N_AGENTS,
    max_iterations=MAX_ITERATIONS,
    exploration_factor=EXPLORATION_FACTOR,
    exploitation_factor=EXPLOITATION_FACTOR
)

best_params, best_loss, convergence = ninja.optimize()

print("Best params:", best_params)


Optimisation subset sizes: 604928 129599
Starting Ninja Optimization Algorithm...

Time limit callback active
Time limit callback active
Time limit callback active
Time limit callback active
Time limit callback active
Time limit callback active

Iteration 1/6
Time limit callback active
Time limit callback active
Time limit callback active
Time limit callback active
Time limit callback active
Time limit callback active
  Best validation loss so far: 1.3614169802167453e-05
  Best hyperparameters so far:
    lstm_layers: 2
    units: 128
    dropout: 0.6
    optimizer: adamw
    learning_rate: 0.0005
    batch_size: 32

Iteration 2/6
Time limit callback active
Time limit callback active
Time limit callback active
Time limit callback active
Time limit callback active
Time limit callback active
  Best validation loss so far: 1.3553126336773857e-05
  Best hyperparameters so far:
    lstm_layers: 2
    units: 64
    dropout: 0.3
    optimizer: adamw
    learning_rate: 0.0005
    batch_size: 3

In [10]:
# Save best params
with open(os.path.join(experiment_dir, "best_params.json"), "w") as f:
    json.dump(to_python_types(best_params), f, indent=4)

# Save convergence
np.save(os.path.join(experiment_dir, "convergence.npy"), convergence)

print("Optimisation artifacts saved.")


Optimisation artifacts saved.


In [15]:
import tensorflow.keras.backend as K
import gc

K.clear_session()
gc.collect()


23143

In [16]:
from tensorflow.keras.callbacks import EarlyStopping

model = create_lstm_model(best_params, seq_len, num_feats)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=FINAL_PATIENCE,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    X_train_seq,
    y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=FINAL_EPOCHS,
    batch_size= 16,                ###############reduced batch size
    callbacks=[early_stop],
    verbose=1
)

# Save model
model.save(os.path.join(experiment_dir, "model", "NiOA_DRNN_k60.h5"))

print("Final model saved.")


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
results_df, y_true, y_pred, explanation = evaluate_model(
    model, X_test_seq, y_test_seq
)

print(results_df)
print(explanation)

# Save predictions
np.save(os.path.join(experiment_dir, "predictions", "y_test.npy"), y_true)
np.save(os.path.join(experiment_dir, "predictions", "y_test_pred.npy"), y_pred)

# Save metrics
with open(os.path.join(experiment_dir, "metrics.json"), "w") as f:
    json.dump(
        dict(zip(results_df["Metric"], results_df["Value"])),
        f,
        indent=4
    )

print("Evaluation artifacts saved.")


In [ ]:
metadata = {
    "horizon": FORECAST_HORIZON,
    "sequence_length": SEQUENCE_LENGTH,
    "train_ratio": TRAIN_RATIO,
    "val_ratio": VAL_RATIO,
    "random_seed": RANDOM_SEED,
    "opt_subset_ratio": OPT_SUBSET_RATIO,
    "opt_iterations": MAX_ITERATIONS,
    "n_agents": N_AGENTS,
    "final_epochs": FINAL_EPOCHS,
    "final_patience": FINAL_PATIENCE
}

with open(os.path.join(experiment_dir, "training_config.json"), "w") as f:
    json.dump(metadata, f, indent=4)

print("Training metadata saved.")


In [17]:
# Predicted vs Actual
plt.figure(figsize=(6, 6))
plt.scatter(y_true, y_pred, alpha=0.5)
plt.plot([y_true.min(), y_true.max()],
         [y_true.min(), y_true.max()],
         linestyle="--")
plt.title("Predicted vs Actual (k=60)")
plt.tight_layout()
plt.savefig(os.path.join(experiment_dir, "plots", "pred_vs_actual.png"))
plt.close()

# Residuals
residuals = y_true - y_pred
plt.figure(figsize=(6, 4))
sns.histplot(residuals, bins=40, kde=True)
plt.title("Residual Distribution")
plt.tight_layout()
plt.savefig(os.path.join(experiment_dir, "plots", "residuals.png"))
plt.close()

# Training curve
plt.figure(figsize=(6, 4))
plt.plot(history.history["loss"], label="Train")
plt.plot(history.history["val_loss"], label="Val")
plt.legend()
plt.title("Training vs Validation Loss")
plt.tight_layout()
plt.savefig(os.path.join(experiment_dir, "plots", "training_curve.png"))
plt.close()

# Convergence
plt.figure(figsize=(6, 4))
plt.plot(convergence, marker="o")
plt.title("NinOA Convergence")
plt.tight_layout()
plt.savefig(os.path.join(experiment_dir, "plots", "ninoa_convergence.png"))
plt.close()

print("All plots saved.")


NameError: name 'y_true' is not defined

<Figure size 600x600 with 0 Axes>